In [0]:
!pip install kaggle

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = ""
os.environ["KAGGLE_KEY"] = ""

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:37<00:00, 122MB/s]


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-561545e5-66b7-483c-9041-23 nogroup 8.4G Jan 10 18:20 2019-Nov.csv
-rwxrwxrwx 1 spark-561545e5-66b7-483c-9041-23 nogroup 5.3G Jan 10 18:22 2019-Oct.csv
-rwxrwxrwx 1 spark-561545e5-66b7-483c-9041-23 nogroup 4.3G Jan 10 18:19 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-561545e5-66b7-483c-9041-23 nogroup 8.4G Jan 10 18:20 2019-Nov.csv
-rwxrwxrwx 1 spark-561545e5-66b7-483c-9041-23 nogroup 5.3G Jan 10 18:22 2019-Oct.csv


In [0]:
%restart_python

In [0]:
from pyspark.sql import functions as F
input_csv = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"

df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(input_csv)
)

display(df.limit(10))
df.printSchema()


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01T00:00:00.000Z,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01T00:00:00.000Z,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01T00:00:01.000Z,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2019-10-01T00:00:01.000Z,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
2019-10-01T00:00:04.000Z,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
2019-10-01T00:00:05.000Z,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
2019-10-01T00:00:08.000Z,view,17300353,2053013553853497655,null,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d
2019-10-01T00:00:08.000Z,view,31500053,2053013558031024687,null,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6
2019-10-01T00:00:10.000Z,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
2019-10-01T00:00:11.000Z,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
df_sel = df.select(
    "event_time", "event_type", "product_id", "category_code", "brand", "price", "user_id"
)

df_filt = df_sel.filter(
    (F.col("event_type") == F.lit("purchase")) & (F.col("price").isNotNull())
)

brand_sales = (
    df_filt.groupBy("brand")
    .agg(
        F.count("*").alias("purchase_count"),
        F.round(F.sum("price"), 2).alias("revenue")
    )
)

top_brands = brand_sales.orderBy(F.col("revenue").desc(), F.col("purchase_count").desc())

display(top_brands.limit(20))


brand,purchase_count,revenue
apple,142873,1.1120926882E8
samsung,172896,4.640753261E7
xiaomi,56616,9194033.29
null,58214,8540601.03
huawei,23501,4883421.74
acer,6882,3576719.52
lg,8727,3387887.96
lucente,11578,3124113.37
sony,6729,2478196.68
oppo,10891,2412959.76


In [0]:
output_dir = "/Volumes/workspace/ecommerce/ecommerce_data/outputs/top_brands_by_revenue_csv"

(
    top_brands
    .coalesce(1)
    .write.mode("overwrite")
    .option("header", True)
    .csv(output_dir)
)

print("Wrote:", output_dir)

Wrote: /Volumes/workspace/ecommerce/ecommerce_data/outputs/top_brands_by_revenue_csv


In [0]:
delta_out = "workspace.ecommerce.top_brands_by_revenue"

(top_brands.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable(delta_out)
)

spark.table(delta_out).display()

brand,purchase_count,revenue
apple,142873,1.1120926882E8
samsung,172896,4.640753261E7
xiaomi,56616,9194033.29
null,58214,8540601.03
huawei,23501,4883421.74
acer,6882,3576719.52
lg,8727,3387887.96
lucente,11578,3124113.37
sony,6729,2478196.68
oppo,10891,2412959.76
